## Identificar los datos atipicos.
1. Metodo estadistico, a traves del z-score. mide la distancia en desviasiones estandar de un punto a la media.
2. Tecnicas de clustering como DBSCAN.
3. Si q< Q1-1.5*IQR Ó q > Q3 + 1.5 * IQR

### Identificar los atipicos con Boxplot
hace referencia al 1.5 veces el rango intercuartilico mas el Q3 y 1.5 menos veces el rango intercuartilico al Q1.

### Lidear con los valores atipicos en la construcción de modelos.

#### Regresiones robustas.
1. **Ransac**: Random Sample Consensus, Muestreo aleatorio, donde se busca incluir mas datos bueno, hace convinaciones para mejorar el modelo. El modelo asume que los valores malos no tienen patrones especificos.
2. **Huber Regressor**. No intenta excluir los valores atipicos como el RANSAC si no que los penaliza, disminuye su influencia en el modelo. Son tratados como atipicos si el valor del error absoluto de su perdida esta por encima de un umbral llamado epsilon, se ha identificado que un valor de epsilon de 1.35 logra un 95% de eficiencia estadistica.

In [6]:
import pandas as pd
df = pd.read_csv('./felicidad_corrupt.csv')
df.tail(5)

,country,rank,score,high,low,gdp,family,lifexp,freedom,generosity,corruption,dystopia
162,H,163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,I,164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,J,165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,K,166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,L,167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
from sklearn.linear_model import (
    RANSACRegressor, # es un metaestimador, por defecto toma a una reg.lineal
    HuberRegressor # el epsilon mayor permite mas valores atipicos
)
# para SVR el gamma deber ser auto
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
X = df.drop(['country','score'], axis=1)
y = df[['score']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

# Estructura de datos para agilizar el entrenamiento de modelos
estimadores = {
    'SVR':SVR(gamma='auto',C=1.0,epsilon=0.1),
    'RANSAC': RANSACRegressor(),
    'HUBER' : HuberRegressor(epsilon=1.35)
}

In [9]:
for name,estimador in estimadores.items():
    estimador.fit(X_train,y_train)
    predictions = estimador.predict(X_test)

    print('---'*50)
    print(name)
    print("MSE: ",mean_squared_error(y_test,predictions))

------------------------------------------------------------------------------------------------------------------------------------------------------
SVR
MSE:  0.07455771301888525
------------------------------------------------------------------------------------------------------------------------------------------------------
RANSAC
MSE:  1.2756393823693603e-19
------------------------------------------------------------------------------------------------------------------------------------------------------
HUBER
MSE:  2.297192346394143e-06


/home/frluengas/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/frluengas/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/frluengas/anaconda3/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
